In [ ]:
from rich import print
import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()
client = OpenAI(
    base_url="https://models.inference.ai.azure.com",
    api_key=os.getenv('GITHUB_TOKEN'),
)

if client:
    print("Sėkmingai prisijungta Prie Github modelių.\n\n")
else:
    print("Prisijungti prie Github modelių nepavyko.\n\n")

In [ ]:
question = "write me haiku about coding."

response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": question,
        }
    ],
    model="gpt-4o",
    # temperature=1,
    max_tokens=4096,
    # top_p=1
)
print(response.choices[0].message.content)
print('\n pirmos užklausos pabaiga \n')

In [1]:
from transformers import AutoTokenizer, AutoModel
import torch

# Load a pre-trained Hugging Face model
model_name = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Input text
text = "This is a test sentence."
inputs = tokenizer(text, return_tensors="pt", truncation=True)
outputs = model(**inputs)

# Take the mean pooling of the model output to create embeddings
embeddings = outputs.last_hidden_state.mean(dim=1)
print("Embeddings:", embeddings)

c:\Users\V.I.I.S\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Embeddings: tensor([[ 4.0734e-01,  2.8004e-01,  2.1713e-02,  5.1135e-01,  3.4228e-02,
         -8.6229e-02, -8.1606e-02, -7.3586e-02,  1.9557e-01,  1.6150e-01,
          5.0413e-01, -2.2729e-01,  3.3268e-02,  1.9821e-01,  9.0419e-02,
         -2.0050e-01,  1.1427e-01, -2.7303e-01, -1.6283e-01,  2.4640e-01,
          3.3489e-01,  2.6473e-01, -4.7299e-02,  1.1451e-01,  9.6627e-02,
          4.6956e-02, -2.8461e-01,  3.5311e-02,  2.2724e-01, -2.1794e-02,
         -2.6963e-01, -2.0099e-02,  3.1292e-01,  2.3231e-01,  8.2247e-02,
         -1.5383e-02,  2.7738e-01,  1.7025e-01, -2.8432e-02,  7.1675e-02,
          5.5939e-02, -5.1937e-01,  9.2315e-02,  1.0672e-01,  5.2500e-02,
          1.8275e-02, -1.5434e-01,  5.1839e-02, -2.3399e-02, -1.3693e-01,
         -2.5404e-01, -3.4109e-01, -2.7812e-01, -6.5877e-02,  2.7457e-02,
          1.1150e-01,  1.7250e-01,  7.2475e-02,  2.4037e-01,  2.0599e-01,
         -1.6714e-01, -1.1771e-01, -3.4561e-01,  4.0167e-01,  7.1960e-01,
          2.6102e-01, -1.9

In [8]:
import requests
import os
from dotenv import load_dotenv

hf_token = os.getenv('HF_API_TOKEN')

# API_URL = "https://api-inference.huggingface.co/models/sentence-transformers/all-MiniLM-L6-v2"
# headers = {"Authorization": f"Bearer hf_token"}

API_URL = "https://api-inference.huggingface.co/models/sentence-transformers/all-MiniLM-L6-v2"
headers = {"Content-Type": "application/json", "Authorization": f"Bearer {hf_token}"}

payload = {
    "query_embedding": embeddings,  # Pass the embedding vector here
    "filter": {"category": "science"},  # Other filters or metadata
    "top_k": 5  # Number of nearest neighbors to retrieve
}

# Make the request
# Convert tensor to list before sending
payload["query_embedding"] = embeddings.detach().numpy().tolist()
response = requests.post(API_URL, headers=headers, json=payload)

# Parse embeddings from the response
if response.status_code == 200:
    print("Response:", response)
else:
    print(f"Error {response.status_code}:", response.text)

Error 403: {"error":"Your auth method doesn't allow you to make inference requests"}


In [ ]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Get the OpenAI API key
openai_key = os.getenv('OPENAI_KEY')

In [ ]:
import openai
from rich import print

def generate_embedding(text, model="text-embedding-3-small", api_key=openai_key):
    """Generate embeddings for a given text using OpenAI's API."""
    client = openai.OpenAI(api_key=api_key)
    response = client.embeddings.create(input=text, model=model)
    return response.data[0].embedding

print(generate_embedding("Hello, world!"))

In [ ]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

# Only keep post title, headers, and content from the full HTML.
bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()

assert len(docs) == 1
print(f"Total characters: {len(docs[0].page_content)}")
print(docs[0].page_content[:500] + "........")